In [4]:
# !git lfs install || true
# !pip -q install --upgrade pip

In [ ]:
!git clone https://github.com/huggingface/diffusers
%cd diffusers
!pip install -e .

In [ ]:
!pip install -r examples/dreambooth/requirements_sana.txt

In [7]:
!pip install -q "peft>=0.14.0"
!pip install -q accelerate transformers datasets huggingface_hub safetensors wandb

In [ ]:
from accelerate.utils import write_basic_config

write_basic_config()
print("Wrote default accelerate config.")

In [ ]:
# This script downloads the Naruto BLIP Captions dataset from Hugging Face
# and saves the images and captions to a local directory.
# Requirements: Run in a Python environment with the following libraries installed:
# pip install datasets pillow

import os
from datasets import load_dataset
from PIL import Image  # For saving images

# Load the dataset from Hugging Face
dataset = load_dataset("lambdalabs/naruto-blip-captions")

# Create a directory to save the dataset
save_dir = "naruto_dataset"
os.makedirs(save_dir, exist_ok=True)

# The dataset has a 'train' split with images and captions
# Iterate through the dataset and save each image and its caption
for idx, item in enumerate(dataset['train']):
    # Save the image
    image_path = os.path.join(save_dir, f"image_{idx:04d}.jpg")
    item['image'].save(image_path)
    
    # Save the caption to a text file
    caption_path = os.path.join(save_dir, f"caption_{idx:04d}.txt")
    with open(caption_path, "w", encoding="utf-8") as f:
        f.write(item['text'])

print(f"Dataset downloaded and saved to '{save_dir}'. Total items: {len(dataset['train'])}")

In [ ]:
save_dir = "naruto_dataset"

# The sentence to append
append_text = " in Naruto Anime style."

# Iterate through all files in the directory
for filename in os.listdir(save_dir):
    # Check if the file is a .txt file
    if filename.endswith(".txt"):
        file_path = os.path.join(save_dir, filename)
        # Open the file in append mode and add the sentence
        with open(file_path, "a", encoding="utf-8") as f:
            f.write(append_text)

print(f"Appended '{append_text}' to all .txt files in '{save_dir}'.")

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
!pip install gdown -q

In [ ]:
import gdown

# Google Drive file ID
file_id = "16NR1_Zvronbla7j2v4W3_xX8lpVNnEFJ"

# URL format using ID
url = f"https://drive.google.com/uc?id={file_id}"

# Output file name
output = "PPE_SANA.zip"

# Download the file
gdown.download(url, output, quiet=False)

print(f"✅ Download completed: {output}")

In [ ]:
# Path to your downloaded zip file
zip_file = "/teamspace/studios/this_studio/PPE_SANA.zip"

# Destination folder to extract
extract_folder = "/teamspace/studios/this_studio/PPE_SANA"

# Unzip the file
!unzip -q {zip_file} -d {extract_folder}

print(f"✅ Zip file extracted to: {extract_folder}")

In [ ]:
%cd diffusers/examples/dreambooth/

In [1]:
!cp /teamspace/studios/this_studio/diffusers/examples/dreambooth/train_dreambooth_lora_sana.py /teamspace/studios/this_studio/

In [ ]:
!accelerate launch train_dreambooth_lora_sana.py \
  --pretrained_model_name_or_path="Efficient-Large-Model/Sana_1600M_1024px_diffusers" \
  --instance_data_dir="/teamspace/studios/this_studio/PPE_SANA/content/street-work-1/valid/images" \
  --output_dir="/teamspace/studios/this_studio/output/trained-sana-lora" \
  --mixed_precision="bf16" \
  --instance_prompt="A factory worker wearing PPE performing tasks with machinery" \
  --max_sequence_length=1024 \
  --resolution=512 \
  --train_batch_size=2 \
  --gradient_accumulation_steps=2 \
  --gradient_checkpointing \
  --cache_latents \
  --lr_scheduler="constant" \
  --learning_rate=5e-5 \
  --max_train_steps=300 \
  --validation_prompt="A worker wearing helmet, gloves, and safety vest performing maintenance on industrial equipment outdoors. The worker is handling insulators and electrical connections on high-voltage equipment, standing on scaffolding with proper PPE observed" \
  --validation_epochs=25 \
  --seed=0 \
  --report_to tensorboard

In [ ]:
!tensorboard --logdir="/teamspace/studios/this_studio/output/trained-sana-lora/logs"